### Import libraries

In [74]:
from IPython.core.display import display
from src.utils.preprocessing import classic_preprocessing,standardize
from src.utils.get_data import import_data, split_experts
from src.utils.train import hyperparameter_tuning_cv
from src.utils.config import *
import pandas as pd
import numpy as np

In [75]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Data

In [76]:
DATA_PATH = '../../data'
X_coarse, y_coarse = import_data(DATA_PATH, segmentation_type='coarse',
                                 drop_user_features=False,
                                 drop_expert=True)

In [77]:
# For each expert separately
X_e, y_e = import_data(DATA_PATH, segmentation_type='coarse',
                                 drop_user_features=False,
                                 drop_expert=False)

In [5]:
display(X_coarse.head())
display(y_coarse.head())

EEPD50_100  EEPD100_150  \
subject                              file_id                            
008ba489-31ad-44d8-856b-fcf72369dc46 0               8.0          8.0   
                                     1               2.0          3.0   
008c1c9e-aeef-40c5-846c-24f1b964f884 0               4.0          4.0   
                                     1               3.0          5.0   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0               6.0          6.0   

                                              EEPD150_200  EEPD200_250  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                7.0          6.0   
                                     1                2.0          3.0   
008c1c9e-aeef-40c5-846c-24f1b964f884 0                4.0          4.0   
                                     1                5.0          6.0   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0                6.0          7.0   

                                              EEPD250_300  EEPD300_350  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                7.0          9.0   
                                     1                2.0          2.0   
008c1c9e-aeef-40c5-846c-24f1b964f884 0                6.0          6.0   
                                     1                4.0          5.0   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0                6.0          7.0   

                                              EEPD350_400  EEPD400_450  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                7.0          7.0   
                                     1                3.0          3.0   
008c1c9e-aeef-40c5-846c-24f1b964f884 0                5.0          5.0   
                                     1                6.0          5.0   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0                6.0          6.0   

                                              EEPD450_500  EEPD500_550  ...  \
subject                              file_id                            ...   
008ba489-31ad-44d8-856b-fcf72369dc46 0                7.0          8.0  ...   
                                     1                3.0          3.0  ...   
008c1c9e-aeef-40c5-846c-24f1b964f884 0                5.0          5.0  ...   
                                     1                5.0          4.0  ...   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0                8.0          6.0  ...   

                                              PSD_225-425  PSD_450-550  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0           0.314239     0.027049   
                                     1           0.093550     0.107780   
008c1c9e-aeef-40c5-846c-24f1b964f884 0           0.160032     0.034572   
                                     1           0.050611     0.050448   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0           0.264409     0.050780   

                                              PSD_1325-1600  PSD_1600-2000  \
subject                              file_id                                 
008ba489-31ad-44d8-856b-fcf72369dc46 0             0.015518       0.030041   
                                     1             0.013997       0.008304   
008c1c9e-aeef-40c5-846c-24f1b964f884 0             0.017697       0.029626   
                                     1             0.036759       0.036765   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0             0.020884       0.008068   

                                              PSD_2500-2900  PSD_3100-3700  \
subject                              file_id                                 
008ba489-31ad-44d8-856b-fcf72369dc46 0             0.047090       0.009558   
                                     1             0.045378       0.007341   
008c1c9e-aeef-40c5-846c-24f1b964f884 0       

Label
subject                              file_id       
008ba489-31ad-44d8-856b-fcf72369dc46 0          1.0
                                     1          1.0
008c1c9e-aeef-40c5-846c-24f1b964f884 0          1.0
                                     1          1.0
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0          1.0

### Preprocessing

In [78]:
X_coarse = classic_preprocessing(X_coarse)

In [79]:
# Save the expert feature for split
expert = X_e.copy().Expert
X_e = classic_preprocessing(X_e)
X_e.Expert = expert.values

### Split expert models

In [80]:
X_e_1, y_e_1, X_e_2, y_e_2, X_e_3, y_e_3 = split_experts(X_e, y_e)

### Grid search

#### 1. Logistic regression

In [81]:
log_results = hyperparameter_tuning_cv(model='logistic', data=X_coarse, labels=y_coarse.Label, cv_k=5,
                                       params=LOGISTIC_PARAMS)

display(log_results)

f1_score  roc_auc_score  accuracy_score
max_iter oversampling                                         
10000    True          0.448542       0.605041        0.609690
         False         0.182934       0.533048        0.728101
100000   True          0.448542       0.605041        0.609690
         False         0.182934       0.533048        0.728101

In [82]:
# Best model parameters

best_log = log_results.iloc[[log_results.reset_index()['f1_score'].idxmax()]]

display(best_log)


,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.448542,0.605041,0.60969


In [83]:
# for each expert
log_results_1 = hyperparameter_tuning_cv(model='logistic', data=X_e_1, labels=y_e_1, cv_k=5,
                                       params=LOGISTIC_PARAMS)
log_results_2 = hyperparameter_tuning_cv(model='logistic', data=X_e_2, labels=y_e_2, cv_k=5,
                                       params=LOGISTIC_PARAMS)
log_results_3 = hyperparameter_tuning_cv(model='logistic', data=X_e_3, labels=y_e_3, cv_k=5,
                                       params=LOGISTIC_PARAMS)

In [84]:
# Best model parameters

best_log_1 = log_results_1.iloc[[log_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_log_2 = log_results_2.iloc[[log_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_log_3 = log_results_3.iloc[[log_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_log_1, best_log_2, best_log_3)


,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.386331,0.662814,0.742671


,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.319455,0.565675,0.605319


,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.645414,0.652769,0.652


#### 2. Linear Discriminant Analysis

In [85]:
lda_results = hyperparameter_tuning_cv(model='lda', data=X_coarse, labels=y_coarse.Label, cv_k=5,
                                       params=LDA_PARAMS)

display(lda_results)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.453318,0.608708,0.609690
False,0.190517,0.536297,0.730426


In [86]:
# Best model parameters

best_lda = lda_results.iloc[[lda_results.reset_index()['f1_score'].idxmax()]]

display(best_lda)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.453318,0.608708,0.60969


In [87]:
# For each expert
lda_results_1 = hyperparameter_tuning_cv(model='lda', data=X_e_1, labels=y_e_1, cv_k=5,
                                       params=LDA_PARAMS)
lda_results_2 = hyperparameter_tuning_cv(model='lda', data=X_e_2, labels=y_e_2, cv_k=5,
                                       params=LDA_PARAMS)
lda_results_3 = hyperparameter_tuning_cv(model='lda', data=X_e_3, labels=y_e_3, cv_k=5,
                                       params=LDA_PARAMS)


In [88]:
best_lda_1 = lda_results_1.iloc[[lda_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_lda_2 = lda_results_2.iloc[[lda_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_lda_3 = lda_results_3.iloc[[lda_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_lda_1, best_lda_2, best_lda_3)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.400696,0.682819,0.729642


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.32806,0.57182,0.593617


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.642754,0.6504,0.649714


#### 3. K-nearest Neighbors

In [89]:
knn_results = hyperparameter_tuning_cv(model='knn', data=X_coarse, labels=y_coarse.Label, cv_k=5,
                                       params=KNN_PARAMS)

display(knn_results)

f1_score  roc_auc_score  accuracy_score
n_neighbors oversampling                                         
1           True          0.460393       0.614081        0.606202
            False         0.428441       0.609254        0.687403
2           True          0.439225       0.607262        0.649806
            False         0.217001       0.537780        0.720736
3           True          0.452611       0.595151        0.538178
            False         0.328887       0.559909        0.686047
4           True          0.446949       0.598711        0.581202
            False         0.214472       0.538665        0.723450
5           True          0.448678       0.582962        0.496512
            False         0.293522       0.552384        0.700969
6           True          0.439974       0.582301        0.533333
            False         0.192196       0.532057        0.721705
7           True          0.442007       0.571764        0.473837
            False         0.253118       0.542212        0.708333
8           True          0.441032       0.579074        0.513178
            False         0.175681       0.529733        0.725388
9           True          0.438355       0.564864        0.459109
            False         0.231525       0.537261        0.711628
10          True          0.436267       0.569315        0.488178
            False         0.169273       0.530330        0.729070
11          True          0.433953       0.556608        0.441667
            False         0.206372       0.534442        0.719574
12          True          0.432928       0.561355        0.467248
            False         0.140645       0.522888        0.727519
13          True          0.429559       0.549276        0.429457
            False         0.174384       0.526566        0.720349
14          True          0.431224       0.556742        0.454264
            False         0.131099       0.522159        0.729845
15          True          0.430366       0.548582        0.422481
            False         0.156689       0.524330        0.723643

In [90]:
# Best model parameters

best_knn = knn_results.iloc[[knn_results.reset_index()['f1_score'].idxmax()]]

display(best_knn)

,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
1,True,0.460393,0.614081,0.606202


In [91]:
knn_results_1 = hyperparameter_tuning_cv(model='knn', data=X_e_1, labels=y_e_1, cv_k=5, params=KNN_PARAMS)
knn_results_2 = hyperparameter_tuning_cv(model='knn', data=X_e_2, labels=y_e_2, cv_k=5, params=KNN_PARAMS)
knn_results_3 = hyperparameter_tuning_cv(model='knn', data=X_e_3, labels=y_e_3, cv_k=5, params=KNN_PARAMS)

In [92]:
best_knn_1 = knn_results_1.iloc[[knn_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_knn_2 = knn_results_2.iloc[[knn_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_knn_3 = knn_results_3.iloc[[knn_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_knn_1, best_knn_2, best_knn_3)

,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
1,True,0.362337,0.657356,0.673616


,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
1,True,0.33767,0.58225,0.605319


,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
1,False,0.642593,0.656776,0.657143


#### 4. Support Vector Classifier

In [93]:
svc_results = hyperparameter_tuning_cv(model='svc', data=X_coarse, labels=y_coarse.Label, cv_k=5,
                                       params=SVC_PARAMS)

display(svc_results)

f1_score  roc_auc_score  accuracy_score
kernel gamma oversampling                                         
linear 0.10  True          0.449880       0.602992        0.594380
             False         0.000000       0.500000        0.733140
rbf    0.10  True          0.134792       0.529799        0.741279
             False         0.018825       0.503859        0.734302
linear 0.01  True          0.449880       0.602992        0.594380
             False         0.000000       0.500000        0.733140
rbf    0.01  True          0.438819       0.603072        0.631395
             False         0.010160       0.502426        0.734302

In [94]:
# Best model parameters

best_svc = svc_results.iloc[[svc_results.reset_index()['f1_score'].idxmax()]]

display(best_svc)

,,,f1_score,roc_auc_score,accuracy_score
kernel,gamma,oversampling,,,
linear,0.1,True,0.44988,0.602992,0.59438


In [95]:
svc_results_1 = hyperparameter_tuning_cv(model='svc', data=X_e_1, labels=y_e_1, cv_k=5, params=SVC_PARAMS)
svc_results_2 = hyperparameter_tuning_cv(model='svc', data=X_e_2, labels=y_e_2, cv_k=5, params=SVC_PARAMS)
svc_results_3 = hyperparameter_tuning_cv(model='svc', data=X_e_3, labels=y_e_3, cv_k=5, params=SVC_PARAMS)

In [96]:
best_svc_1 = svc_results_1.iloc[[svc_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_svc_2 = svc_results_2.iloc[[svc_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_svc_3 = svc_results_3.iloc[[svc_results_3.reset_index()['roc_auc_score'].idxmax()]]


display(best_svc_1, best_svc_2, best_svc_3)

,,,f1_score,roc_auc_score,accuracy_score
kernel,gamma,oversampling,,,
rbf,0.01,True,0.416856,0.671775,0.796743


,,,f1_score,roc_auc_score,accuracy_score
kernel,gamma,oversampling,,,
linear,0.1,True,0.319348,0.562929,0.579787


,,,f1_score,roc_auc_score,accuracy_score
kernel,gamma,oversampling,,,
rbf,0.01,False,0.643629,0.670865,0.671429


#### 5. Naive Bayes Classifier

In [97]:
nb_results = hyperparameter_tuning_cv(model='naive_bayes', data=X_coarse, labels=y_coarse.Label, cv_k=5,
                                       params=NAIVE_BAYES_PARAMS)

display(nb_results)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.016892,0.500060,0.729070
False,0.019617,0.499223,0.727132


In [98]:
# Best model parameters

best_nb = nb_results.iloc[[nb_results.reset_index()['f1_score'].idxmax()]]

display(best_nb)


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
False,0.019617,0.499223,0.727132


In [99]:
nb_results_1 = hyperparameter_tuning_cv(model='naive_bayes', data=X_e_1, labels=y_e_1, cv_k=5, params=NAIVE_BAYES_PARAMS)
nb_results_2 = hyperparameter_tuning_cv(model='naive_bayes', data=X_e_2, labels=y_e_2, cv_k=5, params=NAIVE_BAYES_PARAMS)
nb_results_3 = hyperparameter_tuning_cv(model='naive_bayes', data=X_e_3, labels=y_e_3, cv_k=5, params=NAIVE_BAYES_PARAMS)

In [100]:
best_nb_1 = nb_results_1.iloc[[nb_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_nb_2 = nb_results_2.iloc[[nb_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_nb_3 = nb_results_3.iloc[[nb_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_nb_1, best_nb_2, best_nb_3)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.315347,0.603215,0.703583


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
False,0.305102,0.538422,0.512766


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.262818,0.531607,0.545714


#### 6. Decision Tree

In [101]:
dt_results = hyperparameter_tuning_cv(model='decision_tree', data=X_coarse, labels=y_coarse.Label, cv_k=5,
                                       params=DECISION_TREE_PARAMS)

display(dt_results)

f1_score  roc_auc_score  accuracy_score
max_depth oversampling                                         
3         True          0.395655       0.557239        0.548062
          False         0.098930       0.514578        0.726744
5         True          0.347287       0.553241        0.634496
          False         0.236312       0.540791        0.712791
7         True          0.399993       0.579620        0.635853
          False         0.273586       0.543327        0.692829

In [102]:
# Best model parameters

best_dt = dt_results.iloc[[dt_results.reset_index()['f1_score'].idxmax()]]

display(best_dt)


,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
7,True,0.399993,0.57962,0.635853


In [103]:
dt_results_1 = hyperparameter_tuning_cv(model='decision_tree', data=X_e_1, labels=y_e_1, cv_k=5, params=DECISION_TREE_PARAMS)
dt_results_2 = hyperparameter_tuning_cv(model='decision_tree', data=X_e_2, labels=y_e_2, cv_k=5, params=DECISION_TREE_PARAMS)
dt_results_3 = hyperparameter_tuning_cv(model='decision_tree', data=X_e_3, labels=y_e_3, cv_k=5, params=DECISION_TREE_PARAMS)

In [104]:
best_dt_1 = dt_results_1.iloc[[dt_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_dt_2 = dt_results_2.iloc[[dt_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_dt_3 = dt_results_3.iloc[[dt_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_dt_1, best_dt_2, best_dt_3)

,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
3,True,0.377647,0.659949,0.726384


,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
5,True,0.307556,0.562107,0.659574


,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
3,True,0.603566,0.614556,0.612


#### 7. Random Forest

In [105]:
rf_results = hyperparameter_tuning_cv(model='random_forest', data=X_coarse, labels=y_coarse.Label, cv_k=5,
                                       params=RANDOM_FOREST_PARAMS)

display(rf_results)

f1_score  roc_auc_score  accuracy_score
max_depth n_estimators oversampling                                         
3         3            True          0.372024       0.542396        0.573643
                       False         0.023809       0.503521        0.732558
          5            True          0.397661       0.564212        0.588372
                       False         0.019971       0.502779        0.732364
          7            True          0.401940       0.567809        0.591860
                       False         0.004361       0.500578        0.732946
5         3            True          0.375316       0.550521        0.593798
                       False         0.097244       0.506911        0.716085
          5            True          0.404027       0.574327        0.613566
                       False         0.088075       0.514311        0.731008
          7            True          0.397315       0.572223        0.619767
                       False         0.067554       0.510816        0.731783
7         3            True          0.389289       0.563500        0.606977
                       False         0.175303       0.522734        0.712791
          5            True          0.384358       0.568419        0.633140
                       False         0.143157       0.521899        0.724806
          7            True          0.380496       0.569564        0.644380
                       False         0.113794       0.517656        0.727519

In [106]:
# Best model parameters

best_rf = rf_results.iloc[[rf_results.reset_index()['f1_score'].idxmax()]]

display(best_rf)

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
5,5,True,0.404027,0.574327,0.613566


In [107]:
rf_results_1 = hyperparameter_tuning_cv(model='random_forest', data=X_e_1, labels=y_e_1, cv_k=5, params=RANDOM_FOREST_PARAMS)
rf_results_2 = hyperparameter_tuning_cv(model='random_forest', data=X_e_2, labels=y_e_2, cv_k=5, params=RANDOM_FOREST_PARAMS)
rf_results_3 = hyperparameter_tuning_cv(model='random_forest', data=X_e_3, labels=y_e_3, cv_k=5, params=RANDOM_FOREST_PARAMS)


In [108]:
best_rf_1 = rf_results_1.iloc[[rf_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_rf_2 = rf_results_2.iloc[[rf_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_rf_3 = rf_results_3.iloc[[rf_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_rf_1, best_rf_2, best_rf_3)

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
3,7,True,0.386667,0.66457,0.738762


,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
5,5,True,0.286529,0.550924,0.675


,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
5,7,False,0.600122,0.640187,0.64


#### 8. Gradient Boosting

In [109]:
gb_results = hyperparameter_tuning_cv(model='gradient_boosting', data=X_coarse, labels=y_coarse.Label, cv_k=5,
                                       params=GRADIENT_BOOSTING_PARAMS)

display(gb_results)

f1_score  roc_auc_score  accuracy_score
max_depth n_estimators oversampling                                         
3         3            True          0.356616       0.551925        0.630039
                       False         0.000000       0.500000        0.733140
          5            True          0.357711       0.552820        0.631395
                       False         0.000000       0.500000        0.733140
          7            True          0.363542       0.566797        0.661240
                       False         0.000000       0.500000        0.733140
5         3            True          0.324858       0.551492        0.664147
                       False         0.000000       0.500000        0.733140
          5            True          0.348153       0.566934        0.681008
                       False         0.002944       0.500212        0.732752
          7            True          0.361731       0.575170        0.686628
                       False         0.018510       0.503267        0.733527
7         3            True          0.369090       0.566129        0.650000
                       False         0.001394       0.500217        0.733140
          5            True          0.367523       0.571405        0.668023
                       False         0.033572       0.504237        0.731008
          7            True          0.372350       0.577014        0.678682
                       False         0.068534       0.510740        0.731395

In [110]:
# Best model parameters

best_gb = gb_results.iloc[[gb_results.reset_index()['f1_score'].idxmax()]]

display(best_gb)

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
7,7,True,0.37235,0.577014,0.678682


In [111]:
gb_results_1 = hyperparameter_tuning_cv(model='gradient_boosting', data=X_e_1, labels=y_e_1, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)
gb_results_2 = hyperparameter_tuning_cv(model='gradient_boosting', data=X_e_2, labels=y_e_2, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)
gb_results_3 = hyperparameter_tuning_cv(model='gradient_boosting', data=X_e_3, labels=y_e_3, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)

In [112]:
best_gb_1 = gb_results_1.iloc[[gb_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_gb_2 = gb_results_2.iloc[[gb_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_gb_3 = gb_results_3.iloc[[gb_results_3.reset_index()['roc_auc_score'].idxmax()]]
display(best_gb_1, best_gb_2, best_gb_3)

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
3,3,True,0.388252,0.6726,0.716612


,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
5,3,True,0.259389,0.548126,0.731383


,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
5,5,True,0.609372,0.640726,0.641714


### Results

In [113]:
#display('logistic', best_log)
#display('lda', best_lda)
#display('knn', best_knn)
#display('svc', best_svc)
#display('naive_bayes', best_nb)
#display('decision_tree', best_dt)
#display('random_forest', best_rf)
#display('gradient_boosting', best_gb)

In [114]:
weighted_average = (np.array([len(X_e_1), len(X_e_2), len(X_e_3)]) / len(X_e))

e_log = np.sum(pd.concat([best_log_1,best_log_2,best_log_3]).roc_auc_score.values * weighted_average)
e_lda = np.sum(pd.concat([best_lda_1,best_lda_2,best_lda_3]).roc_auc_score.values * weighted_average)
e_knn = np.sum(pd.concat([best_knn_1,best_knn_2,best_knn_3]).roc_auc_score.values * weighted_average)
e_svc = np.sum(pd.concat([best_svc_1,best_svc_2,best_svc_3]).roc_auc_score.values * weighted_average)
e_nb = np.sum(pd.concat([best_nb_1,best_nb_2,best_nb_3]).roc_auc_score.values * weighted_average)
e_dt = np.sum(pd.concat([best_dt_1,best_dt_2,best_dt_3]).roc_auc_score.values * weighted_average)
e_rf = np.sum(pd.concat([best_rf_1,best_rf_2,best_rf_3]).roc_auc_score.values * weighted_average)
e_gb = np.sum(pd.concat([best_gb_1,best_gb_2,best_gb_3]).roc_auc_score.values * weighted_average)



In [115]:
# Results expert features
# First row is Expert 1, second Expert 2, third Expert 3
#display('logistic', pd.concat([best_log_1,best_log_2,best_log_3]))
#display('lda', pd.concat([best_lda_1,best_lda_2,best_lda_3]))
display('knn', pd.concat([best_knn_1,best_knn_2,best_knn_3]))
#display('svc', pd.concat([best_svc_1,best_svc_2,best_svc_3]))
#display('naive_bayes', pd.concat([best_nb_1,best_nb_2,best_nb_3]))
#display('decision_tree', pd.concat([best_dt_1,best_dt_2,best_dt_3]))
#display('random_forest', pd.concat([best_rf_1,best_rf_2,best_rf_3]))
#display('gradient_boosting', pd.concat([best_gb_1,best_gb_2,best_gb_3]))

'knn'

f1_score  roc_auc_score  accuracy_score
n_neighbors oversampling                                         
1           True          0.362337       0.657356        0.673616
            True          0.337670       0.582250        0.605319
            False         0.642593       0.656776        0.657143

In [116]:
results = pd.DataFrame(data={'models':['logistic', 'lda', 'knn', 'svc', 'naive_bayes', 'decision_tree', 'random_forest', 'gradient_boosting'], 
                   'auc_best': [best_log.roc_auc_score.values[0], best_lda.roc_auc_score.values[0], best_knn.roc_auc_score.values[0], best_svc.roc_auc_score.values[0], 
                                best_nb.roc_auc_score.values[0], best_dt.roc_auc_score.values[0], best_rf.roc_auc_score.values[0], best_gb.roc_auc_score.values[0]],
                             'auc_expert_weighted':[e_log, e_lda, e_knn, e_svc, e_nb, e_dt, e_rf, e_gb]})

display(results)


,models,auc_best,auc_expert_weighted
0,logistic,0.605041,0.624055
1,lda,0.608708,0.631432
2,knn,0.614081,0.629824
3,svc,0.602992,0.631851
4,naive_bayes,0.499223,0.555362
5,decision_tree,0.579620,0.608953
6,random_forest,0.574327,0.614943
7,gradient_boosting,0.577014,0.616493


### Conclusions


In [64]:
# Save dataframe
#results.to_pickle("results_coarse_metadata.pkl")

In [117]:
results.mean(axis = 0)

auc_best               0.582626
auc_expert_weighted    0.614114
dtype: float64

In [118]:
results.loc[results['auc_best'].argmax()]

models                      knn
auc_best               0.614081
auc_expert_weighted    0.629824
Name: 2, dtype: object

In [119]:
results.loc[results['auc_expert_weighted'].argmax()]

models                      svc
auc_best               0.602992
auc_expert_weighted    0.631851
Name: 3, dtype: object

## ---------------------------------------

In [66]:
results.mean(axis = 0)

auc_best               0.575726
auc_expert_weighted    0.604080
dtype: float64

In [71]:
results.loc[results['auc_best'].argmax()]

models                      knn
auc_best                0.61416
auc_expert_weighted    0.630148
Name: 2, dtype: object

In [72]:
results.loc[results['auc_expert_weighted'].argmax()]

models                      knn
auc_best                0.61416
auc_expert_weighted    0.630148
Name: 2, dtype: object